# pystac-client Introduction

This notebook shows the basics of using pystac-client to open an API, iterate through Collections and Items, as well as perform basic searches.

In [ ]:
from pystac_client import Client

# set pystac_client logger to DEBUG to see API calls
import logging
logging.basicConfig()
logger = logging.getLogger('pystac_client')
logger.setLevel(logging.DEBUG)

# Client

We first connect to an API by retrieving the root catalog, or landing page, of the API with the `Client.open` function.

In [ ]:
# STAC API root URL
URL = 'https://planetarycomputer.microsoft.com/api/stac/v1'

# custom headers
headers = []

cat = Client.open(URL, headers=headers)
cat

# Collections

As with a static catalog the `get_collections` function will iterate through the Collections in the Catalog. Notice that because this is an API it can get all the Collections through a single call, rather than having to fetch each one individually.

In [ ]:
for collection in cat.get_collections():
    print(collection)

In [ ]:
collection = cat.get_collection('aster-l1t')
collection

# Items

The main functions for getting items return iterators, where pystac-client will handle retrieval of additional pages when needed. Note that one request is made for the first ten items, then a second request for the next ten.

In [ ]:
items = collection.get_items()

def get_ten_items(items):
    for i, item in enumerate(items):
        print(f"{i}: {item}")
        if i == 9:
            return
        
get_ten_items(items)
get_ten_items(items)

# Searching APIs

More useful than iterating through items in a Catalog or Collection, is the ability to search for Items based on spatial or temporal properties.

In [ ]:
search = cat.search(
    collections = "aster-l1t",
    datetime = "2000-01-01/2010-12-31",
    max_items = 5
)
search

In [ ]:
items = search.get_all_items()
len(items)